In [92]:
import re
import os
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
import random

In [93]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [94]:
driver.get('https://www.daraz.pk/smartphones/')

In [5]:
actions = ActionChains(driver)

In [95]:
productDF = pd.DataFrame(columns=['id','Score','Title','Price','User Rating','Free Shipping Status','Daraz Mall Status','Seller Rating','Shipping rating','link'])

In [96]:
output_csv = 'products.csv'
headers = ['Title', 'Price', 'User Rating', 'Free Shipping Status', 'Daraz Mall Status', 'Seller Rating', 'Shipping rating', 'link']
pd.DataFrame(columns=headers).to_csv(output_csv, index=False)

In [97]:
review_csv = 'reviews.csv'
headers = ['id','Review']
pd.DataFrame(columns=headers).to_csv(review_csv, index=False)

In [98]:
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView();", element)

In [100]:
index = 39
for _ in range(20):
    ad_grid = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    for idx,i in enumerate(ad_cards):
        scroll_to_element(i)
        name = i.find_element(By.XPATH,"//a[@id='id-a-link']")
        if name.text == 'case' or name.text == 'Case' or name.text == 'cover' or name.text == 'Cover' or name.text == 'casing' or name.text == 'Casing' or name.text == 'protector' or name.text == 'Protector':
            continue
        link_element = i.find_element(By.CSS_SELECTOR,"#id-a-link")
        product_url = link_element.get_attribute("href")
        driver.execute_script(f"window.open('{product_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(5)
        try:
            title = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/span")
            price = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span')
            userRating = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]')
            sellerRating = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]')
            shippingRating = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]/div[2]')
            shippingType = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[1]/div/div/div[3]/div/div[1]/div/div[1]/div[1]/div[1]/span[1]')
        except:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        try:
            darazMall = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/img')
            darazMallStatus = 'Yes'
        except:
            darazMallStatus = 'No'
        try:
            row = {}
            row['id'] = index
            row['Title'] = title.text
            row['Price'] = (price.text)
            row['User Rating'] = (userRating.text)
            row['Free Shipping Status'] = shippingType.text
            row['Daraz Mall Status'] = darazMallStatus
            row['Seller Rating'] = (sellerRating.text)
            row['Shipping rating'] = (shippingRating.text)
            row['link'] = product_url
            single_row_df = pd.DataFrame([row])
            single_row_df.to_csv(output_csv, mode='a', header=False, index=False)
            try:
                pages = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[8]/a')
                pages = int(pages.text)
                if pages > 3:
                    pages = 3
                for i in range(pages):
                    body = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]')
                    reviews = body.find_elements(By.CLASS_NAME,'review-item')
                    for j in reviews:
                        try:
                            name = j.find_element(By.CLASS_NAME,'user')
                            review = j.find_element(By.CLASS_NAME,'review-content-sl')
                        except:
                            continue
                        if review.text == '':
                            continue
                        row = {}
                        row['id'] = index
                        row['Review'] = review.text
                        single_row_df = pd.DataFrame([row])
                        single_row_df.to_csv(review_csv, mode='a', header=False, index=False)
                    button = driver.find_element(By.XPATH, '//li[@title="Next Page"]')
                    button.click()
                    time.sleep(2)
            except (NoSuchElementException, StaleElementReferenceException):
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                index += 1
                continue
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            index += 1
        except:
            continue
    nextButton = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div/div/ul/li[9]/a')
    driver.execute_script("arguments[0].scrollIntoView();", nextButton)
    time.sleep(1)
    nextButton.click()
    time.sleep(5)

In [101]:
for _ in range(20):
    ad_grid = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[4]/div[1]/div/div[1]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    for idx,i in enumerate(ad_cards):
        scroll_to_element(i)
        name = i.find_element(By.XPATH,"//a[@id='id-a-link']")
        if name.text == 'case' or name.text == 'Case' or name.text == 'cover' or name.text == 'Cover' or name.text == 'casing' or name.text == 'Casing' or name.text == 'protector' or name.text == 'Protector':
            continue
        link_element = i.find_element(By.CSS_SELECTOR,"#id-a-link")
        product_url = link_element.get_attribute("href")
        driver.execute_script(f"window.open('{product_url}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(5)
        try:
            title = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/span")
            price = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span')
            userRating = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]')
            sellerRating = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]')
            shippingRating = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]/div[2]')
            shippingType = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[1]/div/div/div[3]/div/div[1]/div/div[1]/div[1]/div[1]/span[1]')
        except:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        try:
            darazMall = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/img')
            darazMallStatus = 'Yes'
        except:
            darazMallStatus = 'No'
        try:
            row = {}
            row['id'] = index
            row['Title'] = title.text
            row['Price'] = (price.text)
            row['User Rating'] = (userRating.text)
            row['Free Shipping Status'] = shippingType.text
            row['Daraz Mall Status'] = darazMallStatus
            row['Seller Rating'] = (sellerRating.text)
            row['Shipping rating'] = (shippingRating.text)
            row['link'] = product_url
            single_row_df = pd.DataFrame([row])
            single_row_df.to_csv(output_csv, mode='a', header=False, index=False)
            try:
                pages = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[8]/a')
                pages = int(pages.text)
                if pages > 3:
                    pages = 3
                for i in range(pages):
                    body = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]')
                    reviews = body.find_elements(By.CLASS_NAME,'review-item')
                    for j in reviews:
                        try:
                            name = j.find_element(By.CLASS_NAME,'user')
                            review = j.find_element(By.CLASS_NAME,'review-content-sl')
                        except:
                            continue
                        if review.text == '':
                            continue
                        row = {}
                        row['id'] = index
                        row['Review'] = review.text
                        single_row_df = pd.DataFrame([row])
                        single_row_df.to_csv(review_csv, mode='a', header=False, index=False)
                    button = driver.find_element(By.XPATH, '//li[@title="Next Page"]')
                    button.click()
                    time.sleep(2)
            except (NoSuchElementException, StaleElementReferenceException):
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                index += 1
                continue
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            index += 1
        except:
            continue
    nextButton = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div/div/ul/li[9]/a')
    driver.execute_script("arguments[0].scrollIntoView();", nextButton)
    time.sleep(1)
    nextButton.click()
    time.sleep(5)

In [121]:
productDF = pd.read_csv('products.csv')

In [122]:
productDF['Price'] = productDF['Price'].replace('Rs. ','', regex=True).replace(',','', regex=True).astype(float)
productDF['User Rating'] = productDF['User Rating'].astype(float)
productDF['Seller Rating'] = productDF['Seller Rating'].replace('%','', regex=True)
productDF['Seller Rating'] = pd.to_numeric(productDF['Seller Rating'], errors='coerce')
productDF['Seller Rating'] = productDF['Seller Rating'].fillna(0)
try:
    productDF['Shipping rating'] = productDF['Shipping rating'].replace('%','', regex=True).astype(float)
except:
    productDF['Shipping rating'] = 0


In [123]:
for i in productDF.index:
    Score = productDF.loc[i, 'Price']
    if productDF.loc[i, 'Daraz Mall Status'] == 'Yes':
        Score += 5000
    if productDF.loc[i, 'Free Shipping Status'] == 'Free Shipping':
        Score += 1000
    elif productDF.loc[i, 'Free Shipping Status'] == 'Free Delivery':
        Score += 1000
    elif productDF.loc[i, 'Free Shipping Status'] == 'Get it Today':
        Score += 500
    elif productDF.loc[i, 'Free Shipping Status'] == 'Fastest Delivery':
        Score += 500
    seller_rating = productDF.loc[i, 'Seller Rating']
    if seller_rating == 0 and productDF.loc[i, 'Seller Rating'] == 'New Seller':
        seller_rating = 50
    Score += 10000 * (
        (0.4 * (productDF.loc[i, 'User Rating'] / 5)) +
        (0.4 * (productDF.loc[i, 'Shipping rating'] / 5)) +
        (0.2 * (seller_rating / 100)) +
        0.2 * (productDF.loc[i, 'Shipping rating'] / 100)
    )
    productDF.loc[i, 'Score'] = Score

In [124]:
productDF.to_csv('final_products.csv')

In [125]:
productDF = pd.read_csv('final_products.csv')

In [127]:
productDF['RAM'] = productDF['Title'].str.extract(r'(\d+\s*(?:GB|G|B)\s*(?:RAM)?)')
productDF['RAM'] = productDF['RAM'].str.replace('RAM','')
productDF['RAM'] = productDF['RAM'].str.replace('GB','')
productDF['RAM'] = productDF['RAM'].str.replace('G','')
productDF['RAM'] = productDF['RAM'].fillna(0).astype(int)
productDF['ROM'] = productDF['Title'].str.extract(r'(\d{2,}(?:\s?GB\s?(?:ROM|R))?)')
productDF['ROM'] = productDF['ROM'].str.replace('ROM','')
productDF['ROM'] = productDF['ROM'].str.replace('GB','')
productDF['ROM'] = productDF['ROM'].str.replace('R','')
productDF['ROM'] = productDF['ROM'].fillna(0).astype(int)
productDF['Battery'] = productDF['Title'].str.extract(r'(\d+\s?(?:mAh\s)?Battery)')
productDF['Battery'] = productDF['Battery'].str.replace('mAh','')
productDF['Battery'] = productDF['Battery'].str.replace('Battery','')
productDF['Battery'] = productDF['Battery'].fillna(0).astype(int)


In [130]:
productDF.to_csv('final_products.csv')

In [129]:
productDF = productDF[~productDF.duplicated(keep=False)]